

Note:
This notebook contains all training experiments and hypotheses explored for Part A.
Final selected model weights are saved and used in Tester_PartA.ipynb.



In [1]:
mount = '/content/gdrive'
from google.colab import drive
drive.mount(mount)

Mounted at /content/gdrive


In [2]:
import os

DATA_ROOT = "/content/drive/MyDrive/role_challenge_dataset_ground_truth.csv"

for root, dirs, files in os.walk(DATA_ROOT):
    print(root)
    for f in files[:5]:
        print("  ", f)
    break


In [3]:
import os
import cv2
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

In [4]:
import cv2
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset

def generate_heatmap(size, center, sigma=5):
    x = np.arange(0, size, 1)
    y = np.arange(0, size, 1)
    xx, yy = np.meshgrid(x, y)
    heatmap = np.exp(
        -((xx - center[0])**2 + (yy - center[1])**2) / (2 * sigma**2)
    )
    return heatmap


class FetalLandmarkDataset(Dataset):
    def __init__(self, img_dir, csv_path, img_size=256, sigma=5):
        self.df = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.img_size = img_size
        self.sigma = sigma

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        img_path = f"{self.img_dir}/{row['image_name']}"
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        h, w = img.shape

        img = cv2.resize(img, (self.img_size, self.img_size))
        img = img / 255.0

        landmarks = [
            (row['ofd_1_x'], row['ofd_1_y']),
            (row['ofd_2_x'], row['ofd_2_y']),
            (row['bpd_1_x'], row['bpd_1_y']),
            (row['bpd_2_x'], row['bpd_2_y']),
        ]

        heatmaps = []
        for x, y in landmarks:
            x = int(x * self.img_size / w)
            y = int(y * self.img_size / h)
            heatmaps.append(
                generate_heatmap(self.img_size, (x, y), self.sigma)
            )

        heatmaps = np.stack(heatmaps)

        return (
            torch.tensor(img).unsqueeze(0).float(),
            torch.tensor(heatmaps).float()
        )


In [5]:
dataset = FetalLandmarkDataset(
    img_dir="/content/gdrive/MyDrive/images",
    csv_path="/content/gdrive/MyDrive/role_challenge_dataset_ground_truth.csv"
)

img, heatmaps = dataset[0]

print(img.shape)
print(heatmaps.shape)


torch.Size([1, 256, 256])
torch.Size([4, 256, 256])


In [6]:
import torch.nn as nn

class SimpleUNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 2, stride=2),
            nn.ReLU(),

            nn.ConvTranspose2d(32, 16, 2, stride=2),
            nn.ReLU(),
        )

        self.out = nn.Conv2d(16, 4, 1)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return self.out(x)


In [7]:
dataset = FetalLandmarkDataset(
    img_dir="/content/gdrive/MyDrive/images",
    csv_path="/content/gdrive/MyDrive/role_challenge_dataset_ground_truth.csv",
    img_size=256,
    sigma=5
)


In [8]:
print(len(dataset))
img, heatmaps = dataset[0]
print(img.shape, heatmaps.shape)


622
torch.Size([1, 256, 256]) torch.Size([4, 256, 256])


In [9]:
from torch.utils.data import DataLoader
train_loader = DataLoader(
    dataset,
    batch_size=8,
    shuffle=True,
    num_workers=2
)


In [10]:
import torch.nn as nn
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimpleUNet().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [11]:
for epoch in range(10):
    running_loss = 0.0

    for img, heatmaps in train_loader:
        img = img.to(device)
        heatmaps = heatmaps.to(device)

        pred = model(img)
        loss = criterion(pred, heatmaps)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1} | Loss: {running_loss/len(train_loader):.4f}")


Epoch 1 | Loss: 0.0051
Epoch 2 | Loss: 0.0012
Epoch 3 | Loss: 0.0012
Epoch 4 | Loss: 0.0012
Epoch 5 | Loss: 0.0012
Epoch 6 | Loss: 0.0012
Epoch 7 | Loss: 0.0012
Epoch 8 | Loss: 0.0012
Epoch 9 | Loss: 0.0012
Epoch 10 | Loss: 0.0012


In [12]:
# Sanity check: model output shape
img, gt_heatmaps = dataset[0]
img = img.unsqueeze(0).to(device)

with torch.no_grad():
    pred_heatmaps = model(img)

print("GT heatmaps:", gt_heatmaps.shape)
print("Pred heatmaps:", pred_heatmaps.shape)


GT heatmaps: torch.Size([4, 256, 256])
Pred heatmaps: torch.Size([1, 4, 256, 256])


In [13]:
torch.save(
    model.state_dict(),
    "hypothesis_1_full_saved_model.pth"
)


In [14]:
dataset = FetalLandmarkDataset(
    img_dir="/content/gdrive/MyDrive/images",
    csv_path="/content/gdrive/MyDrive/role_challenge_dataset_ground_truth.csv",
    img_size=256,
    sigma=3
)


In [15]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    dataset,
    batch_size=8,
    shuffle=True,
    num_workers=2
)


In [16]:
model = SimpleUNet().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [17]:
for epoch in range(10):
    running_loss = 0.0

    for img, heatmaps in train_loader:
        img = img.to(device)
        heatmaps = heatmaps.to(device)

        pred = model(img)
        loss = criterion(pred, heatmaps)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1} | Loss: {running_loss/len(train_loader):.4f}")


Epoch 1 | Loss: 0.0057
Epoch 2 | Loss: 0.0004
Epoch 3 | Loss: 0.0004
Epoch 4 | Loss: 0.0004
Epoch 5 | Loss: 0.0004
Epoch 6 | Loss: 0.0004
Epoch 7 | Loss: 0.0004
Epoch 8 | Loss: 0.0004
Epoch 9 | Loss: 0.0004
Epoch 10 | Loss: 0.0004


In [18]:
torch.save(
    model.state_dict(),
    "hypothesis_2_full_saved_model.pth"
)


In [19]:
import cv2
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
def generate_heatmap(size, center, sigma=5):
    x = np.arange(0, size, 1)
    y = np.arange(0, size, 1)
    xx, yy = np.meshgrid(x, y)
    heatmap = np.exp(
        -((xx - center[0])**2 + (yy - center[1])**2) / (2 * sigma**2)
    )
    return heatmap


In [20]:
class FetalLandmarkDataset(Dataset):
    def __init__(self, img_dir, csv_path, img_size=256, sigma=5, augment=False):
        self.df = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.img_size = img_size
        self.sigma = sigma
        self.augment = augment

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        img_path = f"{self.img_dir}/{row['image_name']}"
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        h, w = img.shape

        img = cv2.resize(img, (self.img_size, self.img_size))
        img = img / 255.0

        landmarks = [
            (row['ofd_1_x'], row['ofd_1_y']),
            (row['ofd_2_x'], row['ofd_2_y']),
            (row['bpd_1_x'], row['bpd_1_y']),
            (row['bpd_2_x'], row['bpd_2_y']),
        ]

        flipped = False
        if self.augment and np.random.rand() > 0.5:
            img = np.flip(img, axis=1).copy()
            flipped = True

        heatmaps = []
        for x, y in landmarks:
            x = int(x * self.img_size / w)
            y = int(y * self.img_size / h)

            if flipped:
                x = self.img_size - x

            heatmaps.append(
                generate_heatmap(self.img_size, (x, y), self.sigma)
            )

        heatmaps = np.stack(heatmaps)

        return (
            torch.tensor(img).unsqueeze(0).float(),
            torch.tensor(heatmaps).float()
        )


In [21]:
dataset = FetalLandmarkDataset(
    img_dir="/content/gdrive/MyDrive/images",
    csv_path="/content/gdrive/MyDrive/role_challenge_dataset_ground_truth.csv",
    img_size=256,
    sigma=5,
    augment=True \
)


In [22]:
img, heatmaps = dataset[0]
print(img.shape)        # [1, 256, 256]
print(heatmaps.shape)  # [4, 256, 256]


torch.Size([1, 256, 256])
torch.Size([4, 256, 256])


In [23]:
train_loader = DataLoader(
    dataset,
    batch_size=8,
    shuffle=True,
    num_workers=2
)


In [24]:
class SimpleUNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 2, stride=2),
            nn.ReLU(),
        )

        self.out = nn.Conv2d(16, 4, 1)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return self.out(x)


In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimpleUNet().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [26]:
for epoch in range(10):
    running_loss = 0.0

    for img, heatmaps in train_loader:
        img = img.to(device)
        heatmaps = heatmaps.to(device)

        pred = model(img)
        loss = criterion(pred, heatmaps)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1} | Loss: {running_loss/len(train_loader):.4f}")


Epoch 1 | Loss: 0.0031
Epoch 2 | Loss: 0.0012
Epoch 3 | Loss: 0.0012
Epoch 4 | Loss: 0.0012
Epoch 5 | Loss: 0.0012
Epoch 6 | Loss: 0.0012
Epoch 7 | Loss: 0.0012
Epoch 8 | Loss: 0.0012
Epoch 9 | Loss: 0.0012
Epoch 10 | Loss: 0.0012


In [27]:
torch.save(
    model.state_dict(),
    "hypothesis_final_full_saved_model.pth"
)
